In [ ]:
import os
import sys 
os.chdir(os.path.join(os.getenv('HOME'), 'RPOSE'))
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
sys.path.append(os.path.join(os.getcwd() + '/core'))
sys.path.append(os.path.join(os.getcwd() + '/segmentation'))
import coloredlogs

import coloredlogs
coloredlogs.install()
from collections import OrderedDict
import time
import shutil
import datetime
import argparse
import signal
import yaml
import logging
from pathlib import Path
import copy
from src_utils import file_path, load_yaml, get_neptune_logger, get_tensorboard_logger
import datasets

env_cfg_path = os.path.join('cfg/env', os.environ['ENV_WORKSTATION_NAME']+ '.yml')
exp_cfg_path = "/home/jonfrey/RPOSE/cfg/exp/exp.yml"
exp = load_yaml(exp_cfg_path)
env = load_yaml(env_cfg_path)
test_dataloader = datasets.fetch_dataloader( exp['test_dataset'], env )
    


In [ ]:
model_point_list = test_dataloader.dataset._pcd_cad_list
base_path_list = test_dataloader.dataset._base_path_list
obj_idx_list = test_dataloader.dataset._obj_idx_list
camera_idx_list = test_dataloader.dataset._camera_idx_list

from visu import Visualizer
visu = Visualizer(p_visu='/home/jonfrey/tmp',num_classes=21)

In [ ]:
import numpy as np
import scipy.io as scio
_, idxs = np.unique( np.array( base_path_list ),return_index=True )
idxs.tolist()

from PIL import Image

from ycb.rotations import *
import torch 
from scipy import io as scio


In [ ]:
for j,i in enumerate(idxs):
    p = base_path_list [i]
    if j >0 : break
    meta = scio.loadmat( p+"-meta.mat")
    img_store = np.array( Image.open(p+"-color.png") )
    K = test_dataloader.dataset.K[ str( camera_idx_list[i] ) ]
    
    img1 = copy.deepcopy( img_store )
    for k, obj_index in enumerate(meta['cls_indexes'].tolist()):
        H = np.eye(4)
        H[:3,:4] =  meta['poses'][:, :, k]   
        color = visu.SEG_COLORS[obj_index[0]-1].tolist()
        color[3] = 170
        color = tuple(color)
        img1 = visu.plot_estimated_pose( 
            img = img1, 
            points = model_point_list[obj_index[0]-1], 
            H = H,
            K = K, 
            tag = 'Test_init',
            color = color,
            w=3,
            not_log=True)
        model_points = model_point_list[obj_index[0]-1]
        h_gt = H 

    res = Image.fromarray( np.uint8(img1) )
    display(res)

In [ ]:
from ycb.ycb_helper import get_bb_from_depth, get_bb_real_target, BoundingBox
img_test = copy.copy(img1)
from scipy.spatial.transform import Rotation as R


def expand(bb, h,w):
    bb.tl[0] = bb.tl[0]-h
    bb.tl[1] = bb.tl[1]-w
    bb.br[0] = bb.br[0]+h
    bb.br[1] = bb.br[1]+w
    
def calculate_bb_cone( K, bb, mean):
    points = np.stack( [bb.tl.numpy(), bb.br.numpy() ])
    points = np.concatenate( [points,np.ones((2,1))], axis=1)
    return (np.linalg.inv( K ) @ points.T * mean).T

def rel_h (h1,h2):
  return so3_relative_angle(torch.tensor( h1 ) [:3,:3][None], torch.tensor( h2 ) [:3,:3][None])

def get_adaptive_noise(model_points, h_gt, K, obj_idx = 0, factor=5, rot_deg = 30):
    target_points = model_points @ h_gt[:3,:3].T + h_gt[:3,3]
    bb = get_bb_real_target(torch.from_numpy( target_points[None,:,:] ), K[None])[0]
    h_, w_ = bb.height(), bb.width()
    bb_min = copy.deepcopy( bb)
    bb_max = copy.deepcopy( bb)
    expand ( bb_min, h = -int(h_/factor), w = -int(w_/factor))
    expand ( bb_max, h = int(h_/factor), w = int(w_/factor))
    
    
    mean_dis = np.mean(target_points[:,2])
    mi = calculate_bb_cone(K,bb_min, mean_dis )
    ma = calculate_bb_cone(K,bb_max, mean_dis )
    a1 = mi-ma

    noise = ( a1[0,0], a1[0,1], np.mean( a1[0,:2] ) *1.2)
    
    h_pred_est = np.eye(4)
    h_pred_est[:3,3] = np.random.uniform(low=h_gt[:3,3]-noise, high=h_gt[:3,3]+noise, size=(3))
    
    if obj_idx == 12:
        while True:
            x = R.from_euler('xy', np.random.uniform( -rot_deg,rot_deg,(2) ), degrees=True).as_matrix() @ h_gt[:3,:3]
            if abs(np.degrees( rel_h(h_gt[:3,:3], x))) < rot_deg:
              break
    
    while  True:
        x = special_ortho_group.rvs(3)
        if abs(np.degrees( rel_h(h_gt[:3,:3], x))) < rot_deg:
          break
    h_pred_est[:3,:3] = x
    return h_pred_est

from scipy.stats import special_ortho_group
h_pred_est = get_adaptive_noise(model_points, h_gt, K, obj_idx = 12 ,factor=5, rot_deg = 30 )

print( h_pred_est )
print( h_gt )  
#     def draw(bb,img,c=127):
#         img[max( int( bb.tl[0] ),0) : min( int(bb.br[0]), img.shape[0]-1), 
#             max( int(bb.tl[1]), 0): min( int(bb.br[1]), img.shape[1]-1), : ] = c
#         return img 

#     print(bb_max)
#     print(bb_min)
#     img_test = draw(bb_max,img_test,c=255)
#     img_test = draw(bb,img_test,c=200)
#     img_test = draw(bb_min,img_test,c=100)
#     res = Image.fromarray( np.uint8(img_test) )
#     display(res)

#     print(noise)

In [ ]:
mi1 = calculate_bb_cone2(K,bb, np.mean(target_points[:,2]) ) #mi = calculate_bb_cone2(K,bb_min, np.mean(target_points[:,2]))
mi1

mi2 = calculate_bb_cone2(K,bb_min, np.mean(target_points[:,2]) ) #mi = calculate_bb_cone2(K,bb_min, np.mean(target_points[:,2]))

mi3 = calculate_bb_cone2(K,bb_min, np.mean(target_points[:,2])*fac ) #mi = calculate_bb_cone2(K,bb_min, np.mean(target_points[:,2]))

mi1, mi2, mi3 

In [ ]:
points = np.stack( [bb.tl.numpy(), bb.br.numpy(), np.array( [bb.tl[0],bb.br[1]] ),np.array( [bb.br[0],bb.tl[1]]) ])
points.shape
points = np.concatenate( [points,np.ones((4,1))], axis=1)
points

In [ ]:
def calculate_bb_cone( K, bb):
    points = np.stack( [bb.tl.numpy(), bb.br.numpy(), np.array( [bb.tl[0],bb.br[1]] ),np.array( [bb.br[0],bb.tl[1]] ) ])
    points = np.concatenate( [points,np.ones((4,1))], axis=1)
    p1 = ((np.linalg.inv( K ) @ points.T) * np.min(target_points[:,2]) ).T
    p2 = ((np.linalg.inv( K ) @ points.T) * np.max(target_points[:,2]) ).T
    return np.concatenate( [ p1,p2],axis= 0)

calculate_bb_cone(K,bb)

